In [91]:
import pandas as pd
import keras
import numpy as np

responses = pd.read_csv('data/response_format_cleaned_ds1.csv', sep=';', header=0)
responses.drop(responses.columns[[0]], axis=1, inplace=True)
responses.head()

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,sequence1,seqOne,Dep5words[Word1],...,wor_all_selected,wor_all_selected1,minidep_scale,minidep_diagnose,depression_episodes,miniGAD_scale,miniGAD_symptoms_scale,miniGAD_diagnose,minidiagnose_category,minidiagnose_category_number
0,434.0,2020-08-07 11:46:22,15.0,en,6.593644e+08,2020-08-07 11:38:22,2020-08-07 11:46:22,2.0,1.0,motivated,...,NA NA happy NA NA NA NA NA NA NA NA NA NA care...,happy carefree satisfied ...,0.0,0,0,0,0,0,NoDi,0
1,184.0,2020-08-07 11:58:36,15.0,en,2.803892e+08,2020-08-07 11:34:31,2020-08-07 11:58:36,2.0,1.0,connected,...,anxious NA NA NA NA NA NA NA NA NA NA tense NA...,anxious tense fearful sad fe...,3.0,0,0,8,5,0,NoDi,0
2,330.0,2020-08-07 11:51:54,15.0,en,6.770686e+07,2020-08-07 11:36:32,2020-08-07 11:51:54,1.0,1.0,Yes,...,anxious NA NA NA NA NA worried NA NA NA NA NA ...,anxious worried scared sad mon...,7.0,0,5,9,5,0,NoDi,0
3,630.0,2020-08-07 13:22:42,15.0,en,1.176643e+09,2020-08-07 12:55:26,2020-08-07 13:22:42,3.0,1.0,minor,...,anxious NA NA NA NA concerned NA NA NA NA NA t...,anxious concerned tense scared ...,3.0,0,5,8,5,0,NoDi,0
4,400.0,2020-08-07 12:04:52,15.0,en,1.012492e+09,2020-08-07 11:37:19,2020-08-07 12:04:52,1.0,1.0,family,...,NA NA NA NA NA concerned NA NA NA NA NA tense ...,concerned tense sad tired ...,4.0,0,2,7,4,1,GAD,2


In [92]:
"""
Only for understanding data and visualize a response example.
Prints column name and response of patient at row 0.
"""
for res, col in zip(responses.iloc[0], responses.columns):
    print("{} -> {}".format(col, res))

id -> 434.0
submitdate -> 2020-08-07 11:46:22
lastpage -> 15.0
startlanguage -> en
seed -> 659364415.0
startdate -> 2020-08-07 11:38:22
datestamp -> 2020-08-07 11:46:22
sequence1 -> 2.0
seqOne -> 1.0
Dep5words[Word1] -> motivated
Dep5words[Word2] -> learning
Dep5words[Word3] -> passionate
Dep5words[Word4] -> enthusiastic
Dep5words[Word5] -> happy
Wor5words[SQ01] -> motivated
Wor5words[SQ02] -> enthusiastic
Wor5words[SQ03] -> learning
Wor5words[SQ04] -> exercise
Wor5words[SQ05] -> nutrition
sequence2 -> 3.0
seq2 -> 2.0
Dep5phraseorwords[SQ01] -> happy
Dep5phraseorwords[SQ02] -> eager to learn
Dep5phraseorwords[SQ03] -> restful sleep
Dep5phraseorwords[SQ04] -> motivated
Dep5phraseorwords[SQ05] -> joyful
Wor5phraseorwords[SQ01] -> content
Wor5phraseorwords[SQ02] -> improving
Wor5phraseorwords[SQ03] -> learning
Wor5phraseorwords[SQ04] -> motivated
Wor5phraseorwords[SQ05] -> better than before
sequence3 -> 1.0
seq3 -> 3.0
Deptext -> Over the past two weeks, I have not been depressed.  I hav

In [93]:
"""
Using 5-gram contexts from the database, a co-occurrence (word by word) matrix was set up, 
where the rows contained the 120,000 most common words in the n-gram database and the columns 
consisted of the 10,000 most common words in the n-gram database.

The variable 'space' is a matrix of the semantic space with dimentions reduced to 512.
"""
space = pd.read_csv('data/spaceEnglish1.csv', encoding= 'unicode_escape')
space.set_index('words', inplace=True)
space.drop(space.columns[[0]], axis=1, inplace=True)
space.dropna(inplace=True)

In [11]:
space.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X503,X504,X505,X506,X507,X508,X509,X510,X511,X512
words,,,,,,,,,,,,,,,,,,,,,
was,-0.234071,-0.278211,-0.100658,-0.269570,-0.115498,-0.000038,-0.036835,0.024037,-0.003974,0.006582,...,-0.011414,0.018075,-0.020312,0.001287,0.024483,0.012867,0.021265,0.016368,0.024858,-0.020382
not,-0.283230,-0.338776,-0.141085,-0.243715,-0.236692,-0.033354,-0.099906,0.053253,-0.025582,-0.040372,...,-0.028472,0.048824,-0.025452,0.007828,0.027658,-0.022135,0.023037,0.005371,-0.001482,-0.024063
by,-0.251058,-0.327183,-0.203889,-0.283337,-0.124522,-0.006537,0.015371,0.131667,-0.130597,0.055605,...,-0.001550,0.027915,-0.012646,-0.005019,0.075544,0.014663,0.013489,-0.022636,0.010127,-0.027951
that,-0.281888,-0.346746,-0.171006,-0.266698,-0.208917,-0.019832,-0.035404,0.044301,-0.076601,0.021328,...,0.019319,0.042742,0.001747,0.019198,0.022598,0.034352,0.033282,-0.006843,0.027052,-0.013623
of,-0.256530,-0.335434,-0.229791,-0.256070,-0.120020,0.017080,0.078004,0.112134,-0.073805,0.098183,...,0.012012,0.005470,-0.022383,-0.042172,-0.003430,-0.013435,0.003697,-0.022769,0.024873,0.010061


In [94]:
"""
Controlling for artifacts relating to frequently occurring words.

1) Calculate, from Google N-gram, a frequency weighted average of all semantic representations in the space.
   (So that the weighting is proportional to how frequently the words occur in Google N-gram.)
2) Subtract this mean prior to aggregating each word, and then add to the final value.
"""

def aggregating_words(responses):
    res_arr = np.zeros(512)
    #mean_array=np.array(responses).astype(np.float)
    #space_mean = np.mean(mean_array, axis=0)
    
    for word in responses:
        word_arr = np.array(space.loc[word])
        res_arr = res_arr + word_arr# - space_mean
    
    #res_arr += space_mean    
    res_arr = res_arr / res_arr.sum() # Normalizing aggregated vector
    return res_arr

In [95]:
"""
Cleans the string from punctuations and removes all words which are not represented in the semantic space. 
"""

import re

words_in_space = set(space.index.values)

def clean_text(text):
    try:
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = list(set(text.split()))
        cleaned_words = [w for w in text if w in words_in_space]
        return cleaned_words
    except Exception as e: 
        print(e)
    

In [97]:
"""
For one row extracts the string of words/text from the important columns. 
The strings are cleaned using the clean_text function and a list of all words (without duplicates) is returned.
"""

def extract_words_dep(inx):
    important_columns = ['Deptext','dep_all_phraces', 'dep_all_words',  'dep_all_selected1']
    all_responded_words_dep = []
    res = responses.iloc[inx]
    for column_name in important_columns:
        words_in_column = res[column_name]
        if isinstance(words_in_column, str): 
            words = clean_text(words_in_column)
            for word in words:
                if word not in all_responded_words_dep:
                    all_responded_words_dep.append(word)
    return all_responded_words_dep

def extract_words_anx(inx):
    important_columns = ['Wortext', 'wor_all_phraces', 'wor_all_words',  'wor_all_selected1']
    all_responded_words_anx = []
    res = responses.iloc[inx]
    
    for column_name in important_columns:
        words_in_column = res[column_name]
        if isinstance(words_in_column, str): 
            words = clean_text(words_in_column)
            for word in words:
                if word not in all_responded_words_anx:
                    all_responded_words_anx.append(word)
    return all_responded_words_anx

In [98]:
"""
Goes through all rows in response dataframe and for each person, converts all the words to an aggregated vector
and adds this to a dictionary to later be converted to a dataframe. 
"""

all_responses_space_dep = {}
all_responses_space_anx = {}

for i in range(len(responses.index)):
    words_dep = extract_words_dep(i)
    words_anx = extract_words_anx(i)
    try:
        all_responses_space_dep[i] = aggregating_words(words_dep)
        all_responses_space_anx[i] = aggregating_words(words_anx)
    except Exception as e:
        print(e)

In [114]:
"""
Creates a dataframe with dimension (nbr of participants x 512).
Each row stores the aggregated semantic vector representations of one persons responses.

This matrix can then be used for the analysis steps. 
"""
vectors_dep = list(all_responses_space_dep.values())
vectors_anx=list(all_responses_space_anx.values())
cols = list(space.columns)
response_space_dep = pd.DataFrame(vectors_dep, columns = cols) 
response_space_anx=pd.DataFrame(vectors_anx, columns = cols) 

### Important variables 

Deptext = Depression text-response <br> 
Wortext = Worry text-response

dep_all_phraces = Depression all phraces responses <br> 
wor_all_phraces = Worry all phraces responses

dep_all_word = Depression all descriptive word responses <br>
wor_all_words = Worry all descriptive word responses

dep_all_selected1 = All selected depression word responses <br>
wor_all_selected1 = All selected worry word responses

<hr style="border:1px solid gray"> </hr>

CESDtot = Center for Epidemiological Studies Depression (CESD) <br>
PHQtot = PHQ-9 = Patient Helath Questionnaire = a depression scale

GADtot = GAD-7 = Generalized anxiety disorder scale <br>
PSWQtot = Penn State Worry Questionniare 

<hr style="border:1px solid gray"> </hr>

miniGAD_diagnose = Self-reported MINI (structured interview) GAD diangose <br>
minidep_diagnose = Self-reported MINI (structured interview) MDD (depression) diangose

# Using the Semantic Representations in Analyses

In [70]:

resp_dep_scale=responses[['minidep_scale']]
resp_GAD_scale=responses[['miniGAD_scale']]
#Checking for NaN values
resp_dep_scale.isnull().values.any() #true
resp_GAD_scale.isnull().values.any() #false

y_anxiety= resp_GAD_scale.values
y_dep=resp_dep_scale.values 

#Replaceing NaN values with mean value of column - perhaps we should do this differently
col_mean = np.nanmean(y_dep, axis=0)
col_mean=np.around(col_mean, decimals=0, out=None) #rounding 
#Find indices that you need to replace
inds = np.where(np.isnan(y_dep))
#Place column means in the indices. Align the array using take
y_dep[inds] = np.take(col_mean, inds[1])


In [113]:
"""
Semantic-numeric correlations. 
Analyzing the relationship between semantic responses and a numerical variable
"""

"""Predicting the corresponding numeric rating scales on the basis of these representations by means 
of multiple linear regression analyses """

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

regr = LinearRegression()

#x_dep = response_space_dep
#x_anx = response_space_anx
#splitting data into training and testing dataset

#X_train_dep, X_test_dep, y_train_dep, y_test_dep = train_test_split(x_dep, y_dep, test_size=0.2,random_state=0)
#regr.fit(X_train_dep, y_train_dep) 

# Predicting the test set results
#y_pred_dep = regr.predict(X_test_dep)

In [ ]:
"""When the predicted variable is categorical, multinomial logistic regression is used."""

